## Imports

In [9]:
import sys
import json
from pprint import pprint
from pathlib import Path
import requests
src_path = Path("../src/").resolve()
sys.path.append(str(src_path))

In [ ]:
from sqlmodel import Session, select
from api.db.session import engine
from api.events.models import EventModel
from sqlalchemy import func
from timescaledb.hyperfunctions import time_bucket
from datetime import datetime, timedelta, timezone
# from timescaledb.hyperfunctions import  time_bucket_gapfill



# base_url = "http://localhost:8000" # local


base_url = "http://localhost:8002"  # docker

### Check API Health


In [11]:
path = "/healthz"
endpoint = f"{base_url}{path}"
print(endpoint)
response = requests.get(endpoint)
print(f"Health: {response.ok}")

http://localhost:8002/healthz
Health: True


## Get Data


### All Events


In [ ]:
path = "/api/events"
endpoint = f"{base_url}{path}"
print(endpoint)
response = requests.get(endpoint)
print(f"status code {response.status_code}")
if response.ok:
    data = response.json()
    pprint(data)
    # for row in data.get("results"):
    #     print(row)

### Simple Data Types: Single Event


In [ ]:
# path = "/api/events/1"
path = "/api/events/5"
endpoint = f"{base_url}{path}"


print(endpoint)
r = requests.get(endpoint)
print(r.ok, r.status_code)
if r.ok:
    data = r.json()
    print(data)

### List Data Types: Multiple Events


In [ ]:
path = "/api/events"
endpoint = f"{base_url}{path}"
response = requests.get(endpoint)
print(endpoint, response.status_code)
if response.ok:
    data = response.json()
    print(data)

### Simple Data Types: Post Event

Not a valid json data


In [ ]:
path = "/api/events"
endpoint = f"{base_url}{path}"
# print(endpoint)
# payload = {"page": "/test+"}
payload = {"id": 27}
response = requests.post(endpoint, data=payload)
print(response.headers)
print(response.text)
if response.ok:
    data = response.json()
    print(data)

Valid json data


In [ ]:
# import json

payload = {"page": "/test+"}
headers = {"Content-Type": "application/json"}

response = requests.post(endpoint, data=json.dumps(payload), headers=headers)

# Print the response
print(response.status_code)
# print(response.headers) -> still receiving 'content-type': 'application/json', even if the headers changed to xml type
print(response.headers)
print(response.text)

In [ ]:
# import requests
# import xml.etree.ElementTree as ET

# # API endpoint
# url = "http://localhost:8000/api/events"

# # Create the XML payload
# root = ET.Element("event")
# page = ET.SubElement(root, "page")
# page.text = "/test+"

# # Convert the XML to a string
# xml_data = ET.tostring(root, encoding="utf8").decode("utf8")

# # Set the headers
# headers = {"Content-Type": "application/xml"}

# # Send the POST request
# response = requests.post(url, data=xml_data, headers=headers)

# # Print the response
# print(response.status_code)
# print(response.headers)
# print(response.text)

## Send Data to the API


### Create Event Data


In [ ]:
path = "/api/events/"
create_endpoint = f"{base_url}{path}"

# response = requests.post(create_endpoint, json={"page": "/my_new_webpage"})
# response = requests.post(create_endpoint, json={"page": "/my_second_page"})
response = requests.post(create_endpoint, json={"page": "/my_third_page"})

# print(response.status_code)
# print(response.headers)
# print(response.text)
if response.ok:
    data = response.json()
    print(data, type(data))
else:
    print(response.text)

### Update Event Data


In [ ]:
detail_path = "/api/events/3"
detail_endpoint = f"{base_url}{detail_path}"
r = requests.put(detail_endpoint, json={
                 "description": "inline event test", "id": 555})
print(r.ok, r.status_code)
# print(r.headers)
if r.ok:
    data = r.json()
    print(type(data), data)
else:
    print(response.text)

## Delete Event Data


In [ ]:
detail_path = "/api/events/6"
detail_endpoint = f"{base_url}{detail_path}"
r = requests.delete(detail_endpoint)
print(r.ok, r.status_code)
if r.ok:
    data = r.json()
    print(type(data), data)
else:
    print(response.text)
    print(r.headers)

## Send multiple datasets to the API


In [ ]:
import random

path = "/api/events/"
create_endpoint = f"{base_url}{path}"

event_count = 10_000
pages = ["/about", "/contact", "/pages", "/pricing"]

for i in range(event_count):
    page = random.choice(pages)
    response = requests.post(create_endpoint, json={"page": page})
    if response.ok:
        if i % 500 == 0:
            data = response.json()
            print(data, type(data), data.get("items"))
    else:
        print(response.text)

## SQLModel Queries

Outside Fastapi


In [ ]:
# import sys
# from pathlib import Path

# src_path = Path("../src/").resolve()
# sys.path.append(str(src_path))

# from sqlmodel import Session, select
# from pprint  import pprint
# from api.db.session import engine
# from api.events.models import EventModel

In [ ]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.desc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query, end="\n\n")
    print(str(query))
    results = session.exec(query).all()
    pprint(results)

## Aggregate Data With Time Buckets


In [ ]:
from sqlalchemy import func
from timescaledb.hyperfunctions import time_bucket
# from timescaledb.hyperfunctions import  time_bucket_gapfill
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    # bucket width and time field in the db table
    bucket = time_bucket("1 day", EventModel.time)
    # bucket = time_bucket("1 hour", EventModel.time)
    # bucket = time_bucket("1 minute", EventModel.time)
    # pages = ["/about", "/contact","/pages","/pricing"]
    pages = ["/about", "/contact", "/pages"]
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query = (
        select(bucket, EventModel.page, func.count().label("event_count"))
        .where(
            EventModel.time > start,
            EventModel.time <= finish,
            EventModel.page.in_(pages)
        )
        .group_by(bucket, EventModel.page)
        .order_by(bucket, EventModel.page)

    )
    # compiled_query = query.compile(compile_kwargs={"literal_binds":True})
    # print(compiled_query)
    results = session.exec(query).fetchall()
    pprint(results)

## Get Data after Change


### All Events 
Method changed: timebuckets and pages functionalities added


In [ ]:
path = "/api/events"
endpoint = f"{base_url}{path}"
print(endpoint)
# myparams = None
# myparams = {"duration": "30 minutes"}
# myparams = {"duration": "5 minutes", "pages": ["/about", "/contact"]}
myparams = {"pages": ["/about", "/contact", "/pricing", "/nonexistent"]}
response = requests.get(endpoint, params=myparams)
print(f"status code {response.status_code}")
if response.ok:
    data = response.json()
    pprint(data)

http://localhost:8002/api/events
status code 200
[{'bucket': '2025-06-17T00:00:00Z', 'count': 2464, 'page': '/about'},
 {'bucket': '2025-06-17T00:00:00Z', 'count': 2541, 'page': '/contact'},
 {'bucket': '2025-06-17T00:00:00Z', 'count': 2479, 'page': '/pricing'},
 {'bucket': '2025-06-18T00:00:00Z', 'count': 2557, 'page': '/about'},
 {'bucket': '2025-06-18T00:00:00Z', 'count': 2435, 'page': '/contact'},
 {'bucket': '2025-06-18T00:00:00Z', 'count': 2520, 'page': '/pricing'}]
